## Exploration - Data Mining Cup 2022

In [1]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 200)

In [2]:
## Reading in the data sets

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-448-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

file_key = 'orders_items_parent.csv'
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

df = pd.read_csv(file_content_stream)
df.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,category_9,category_10,category_11,category_12,category_13,category_14,category_15,category_16,category_17,category_18,category_19,category_20,category_21,category_22,category_23,category_24,category_25,category_26,category_27,category_28,category_29,category_30,category_31,category_32,category_33,category_34,category_35,parent_category_1,parent_category_2,parent_category_3,parent_category_4,parent_category_5,parent_category_6,parent_category_7,parent_category_8,parent_category_9,parent_category_10,parent_category_11,parent_category_12,parent_category_13,parent_category_14,parent_category_15,parent_category_16,parent_category_17,parent_category_18,parent_category_19,parent_category_20,parent_category_21,parent_category_22
0,2020-06-01,38769,3477,1,186,6,0,196,0,45,74.0,4109.0,3867.0,803.0,4053.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3056.0,3898.0,1543.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-06-01,42535,30474,1,193,10,3,229,3,132,3459.0,3738.0,679.0,1628.0,4072.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3056.0,2824.0,661.0,621.0,3898.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-06-01,42535,15833,1,1318,4,1,455,0,108,2973.0,2907.0,2749.0,3357.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,566.0,1853.0,2045.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-06-01,42535,20131,1,347,4,0,291,3,44,30.0,1515.0,1760.0,2932.0,1287.0,2615.0,3727.0,2450.0,990.0,1395.0,1498.0,2654.0,3663.0,3284.0,1060.0,3268.0,2950.0,1179.0,278.0,2167.0,1763.0,3935.0,3936.0,2925.0,2844.0,377.0,2104.0,3670.0,1482.0,3457.0,767.0,1793.0,4114.0,NaN,NaN,1682.0,621.0,2762.0,2134.0,2231.0,3726.0,264.0,3781.0,3233.0,1396.0,2965.0,2622.0,1812.0,1565.0,1557.0,861.0,3898.0,3819.0,NaN,NaN,NaN,NaN
4,2020-06-01,42535,4325,1,539,6,0,303,0,45,3104.0,1772.0,2029.0,1274.0,3915.0,888.0,1118.0,3882.0,4025.0,4026.0,4068.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1852.0,1682.0,2231.0,3093.0,3898.0,452.0,2720.0,264.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
## Getting date to datetime

df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')

In [4]:
df.shape

(1071538, 67)

In [5]:
df.columns

Index(['date', 'userID', 'itemID', 'order', 'brand', 'feature_1', 'feature_2',
       'feature_3', 'feature_4', 'feature_5', 'category_1', 'category_2',
       'category_3', 'category_4', 'category_5', 'category_6', 'category_7',
       'category_8', 'category_9', 'category_10', 'category_11', 'category_12',
       'category_13', 'category_14', 'category_15', 'category_16',
       'category_17', 'category_18', 'category_19', 'category_20',
       'category_21', 'category_22', 'category_23', 'category_24',
       'category_25', 'category_26', 'category_27', 'category_28',
       'category_29', 'category_30', 'category_31', 'category_32',
       'category_33', 'category_34', 'category_35', 'parent_category_1',
       'parent_category_2', 'parent_category_3', 'parent_category_4',
       'parent_category_5', 'parent_category_6', 'parent_category_7',
       'parent_category_8', 'parent_category_9', 'parent_category_10',
       'parent_category_11', 'parent_category_12', 'parent_category_13'

In [6]:
## Appending week number of transaction

df['week_number'] = int(0)

for i in range(0, df.shape[0]):
    
    ## Extracting day of the month and dividing by 7 to get week number
    df.at[i, 'week_number'] = int(df['date'][i].day / 7) + 1
    
    ## If the week number is greater then 4, make it 4
    if (df.at[i, 'week_number'] > 4):
        df.at[i, 'week_number'] = 4

In [7]:
df['week_number'].value_counts()

4    374097
2    248852
3    243834
1    204755
Name: week_number, dtype: int64

#### Looking at individual customer transaction history

In [8]:
df[df['userID'] == 44176][['date', 'userID', 'itemID', 'order', 'brand', 'week_number']]

,date,userID,itemID,order,brand,week_number
52,2020-06-01,44176,20159,1,827,1
70342,2020-06-17,44176,2109,1,704,3
70343,2020-06-17,44176,13574,2,1180,3
70344,2020-06-17,44176,15971,1,361,3
75732,2020-06-18,44176,8779,1,539,3
295711,2020-08-09,44176,25077,1,186,2
295712,2020-08-09,44176,32243,1,186,2
295713,2020-08-09,44176,1841,1,1065,2
326091,2020-08-17,44176,6065,1,615,3
363675,2020-08-26,44176,23334,1,1496,4


In [9]:
df[df['userID'] == 44176][['date', 'userID', 'itemID', 'order', 'brand', 'week_number']].groupby('itemID')['order'].sum()

itemID
1841     1
2109     2
2492     1
6065     4
6462     2
7035     1
8609     1
8779     7
10648    1
13574    4
13675    1
14732    1
15445    3
15971    2
17068    1
17419    1
17560    1
18430    4
20159    1
21930    1
23334    2
25077    1
25158    1
26271    1
28522    1
29151    1
32204    3
32243    1
32422    1
Name: order, dtype: int64

In [14]:
df[(df['userID'] == 44176) & (df['itemID'] == 8779)][['date', 'userID', 'itemID', 'order', 'brand', 'week_number']]

,date,userID,itemID,order,brand,week_number
75732,2020-06-18,44176,8779,1,539,3
363676,2020-08-26,44176,8779,1,539,4
540987,2020-10-09,44176,8779,1,539,2
610158,2020-10-25,44176,8779,1,539,4
771684,2020-12-01,44176,8779,1,539,1
890815,2020-12-28,44176,8779,1,539,4
1040100,2021-01-26,44176,8779,1,539,4


In [ ]:
pd.DataFrame(df['userID'].unique()).head(20)